In [ ]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os




In [10]:
def get_dataloaders(data_dir, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])

    train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform)
    val_dataset   = datasets.ImageFolder(os.path.join(data_dir, "val"), transform)
    test_dataset  = datasets.ImageFolder(os.path.join(data_dir, "test"), transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader, len(train_dataset.classes)

In [11]:
data_dir = r"D:\STAI_PROJECT\github\Dangerous-Farm-Insects-Classification-main\data\processed\farm_insects\splits" #changable
train_loader, val_loader, test_loader, num_classes = get_dataloaders(data_dir)

In [5]:
model = models.alexnet(pretrained=True)

C:\Users\dell\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\dell\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\dell/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth


100%|██████████| 233M/233M [03:02<00:00, 1.34MB/s] 


In [12]:
model.classifier[6] = nn.Linear(in_features=4096, out_features=num_classes)

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.classifier.parameters(), lr=0.001, momentum=0.9)


for param in model.features.parameters(): #freeze to avoid overfitting
    param.requires_grad = False


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    print("-" * 10)
    
    for phase, loader in [('train', train_loader), ('val', val_loader)]:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(loader.dataset)
        epoch_acc = running_corrects.double() / len(loader.dataset)
        print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")



Epoch 1/10
----------


c:\ProgramData\anaconda3\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


train Loss: 1.9939 Acc: 0.3796
val Loss: 1.2758 Acc: 0.6522
Epoch 2/10
----------
train Loss: 0.9708 Acc: 0.6938
val Loss: 1.1456 Acc: 0.6443
Epoch 3/10
----------
train Loss: 0.6558 Acc: 0.8067
val Loss: 1.0891 Acc: 0.6957
Epoch 4/10
----------
train Loss: 0.4600 Acc: 0.8751
val Loss: 1.0649 Acc: 0.6522
Epoch 5/10
----------
train Loss: 0.3926 Acc: 0.9009
val Loss: 1.1230 Acc: 0.6482
Epoch 6/10
----------
train Loss: 0.3075 Acc: 0.9296
val Loss: 1.0607 Acc: 0.6877
Epoch 7/10
----------
train Loss: 0.2281 Acc: 0.9495
val Loss: 1.1322 Acc: 0.6561
Epoch 8/10
----------
train Loss: 0.2002 Acc: 0.9604
val Loss: 1.1377 Acc: 0.6640
Epoch 9/10
----------
train Loss: 0.1788 Acc: 0.9673
val Loss: 1.1216 Acc: 0.6640
Epoch 10/10
----------
train Loss: 0.1429 Acc: 0.9752
val Loss: 1.1393 Acc: 0.6680


In [14]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 68.35%


In [ ]:
torch.save(model.state_dict(), "alexnet_finetuned_last.pth")

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.classifier.parameters(), lr=0.001, momentum=0.9)


#train everything 

# for param in model.features.parameters(): #freeze to avoid overfitting
#     param.requires_grad = False


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    print("-" * 10)
    
    for phase, loader in [('train', train_loader), ('val', val_loader)]:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(loader.dataset)
        epoch_acc = running_corrects.double() / len(loader.dataset)
        print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")



Epoch 1/10
----------
train Loss: 0.1341 Acc: 0.9742
val Loss: 1.1922 Acc: 0.6561
Epoch 2/10
----------
train Loss: 0.1336 Acc: 0.9703
val Loss: 1.1443 Acc: 0.6719
Epoch 3/10
----------
train Loss: 0.1267 Acc: 0.9722
val Loss: 1.1788 Acc: 0.6680
Epoch 4/10
----------
train Loss: 0.1055 Acc: 0.9822
val Loss: 1.1757 Acc: 0.6561
Epoch 5/10
----------
train Loss: 0.0895 Acc: 0.9812
val Loss: 1.2073 Acc: 0.6719
Epoch 6/10
----------
train Loss: 0.1053 Acc: 0.9772
val Loss: 1.1881 Acc: 0.6640
Epoch 7/10
----------
train Loss: 0.0972 Acc: 0.9802
val Loss: 1.2478 Acc: 0.6838
Epoch 8/10
----------
train Loss: 0.0912 Acc: 0.9832
val Loss: 1.2050 Acc: 0.6561
Epoch 9/10
----------
train Loss: 0.1013 Acc: 0.9722
val Loss: 1.1936 Acc: 0.6680
Epoch 10/10
----------
train Loss: 0.0761 Acc: 0.9802
val Loss: 1.2472 Acc: 0.6719


In [17]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 68.35%


In [18]:
torch.save(model.state_dict(), "alexnet_finetuned_all.pth")

In [20]:


for name, param in model.features.named_parameters():
    if int(name.split('.')[0]) < 8:   # freeze layers before index 8
        param.requires_grad = False
    else:
        param.requires_grad = True


criterion = nn.CrossEntropyLoss()


optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()),
                      lr=0.0005, momentum=0.9)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)


num_epochs = 10
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    print("-" * 10)
    
    for phase, loader in [('train', train_loader), ('val', val_loader)]:
        model.train() if phase == 'train' else model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(loader.dataset)
        epoch_acc = running_corrects.double() / len(loader.dataset)
        print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")


Epoch 1/10
----------
train Loss: 0.9725 Acc: 0.6957
val Loss: 1.2064 Acc: 0.6443
Epoch 2/10
----------
train Loss: 0.7537 Acc: 0.7661
val Loss: 1.1675 Acc: 0.6561
Epoch 3/10
----------
train Loss: 0.5807 Acc: 0.8236
val Loss: 1.1482 Acc: 0.6522
Epoch 4/10
----------
train Loss: 0.4237 Acc: 0.8771
val Loss: 1.1355 Acc: 0.6680
Epoch 5/10
----------
train Loss: 0.3347 Acc: 0.9098
val Loss: 1.1430 Acc: 0.6798
Epoch 6/10
----------
train Loss: 0.2560 Acc: 0.9415
val Loss: 1.1816 Acc: 0.6482
Epoch 7/10
----------
train Loss: 0.2107 Acc: 0.9495
val Loss: 1.2191 Acc: 0.6561
Epoch 8/10
----------
train Loss: 0.1870 Acc: 0.9544
val Loss: 1.2389 Acc: 0.6482
Epoch 9/10
----------
train Loss: 0.1438 Acc: 0.9683
val Loss: 1.2811 Acc: 0.6522
Epoch 10/10
----------
train Loss: 0.1310 Acc: 0.9683
val Loss: 1.2401 Acc: 0.6917


In [21]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 69.30%


In [22]:
torch.save(model.state_dict(), "alexnet_finetuned_partial.pth")